In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd "/content/drive/MyDrive/experiments/MedMNIST2D"


/content/drive/MyDrive/experiments/MedMNIST2D


In [3]:
pip install -r requirements.txt

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 74.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 69.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.7/14.7 MB 50.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 54.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 63.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.2/179.2 kB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.6/97.6 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 62.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 59.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 11.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
import sys
sys.path.append('/content/drive/MyDrive/experiments/MedMNIST2D/models')


In [5]:
import argparse
import os
import time
from collections import OrderedDict
from copy import deepcopy

import medmnist
import numpy as np
import PIL
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision.transforms as transforms
from medmnist import INFO, Evaluator
from models import ResNet18, ResNet50
from torchvision.models import resnet18, resnet50
from tqdm import tqdm

ImportError: cannot import name 'ResNet18' from 'models' (/content/drive/MyDrive/experiments/MedMNIST2D/models/__init__.py)

See the number of channels in each dataset

In [ ]:
data_list=['pathmnist','chestmnist','dermamnist','octmnist','pneumoniamnist','retinamnist','breastmnist','bloodmnist','tissuemnist','organamnist','organcmnist','organsmnist']


for data_flag in data_list:
    info = INFO[data_flag]
    n_channels = info['n_channels']
    print(f'{data_flag}: {n_channels} channels')
task = info['task']

n_classes = len(info['label'])

pathmnist: 3 channels
chestmnist: 1 channels
dermamnist: 3 channels
octmnist: 1 channels
pneumoniamnist: 1 channels
retinamnist: 3 channels
breastmnist: 1 channels
bloodmnist: 3 channels
tissuemnist: 1 channels
organamnist: 1 channels
organcmnist: 1 channels
organsmnist: 1 channels


see the labels from one particular dataset

In [ ]:
info=INFO["pathmnist"]
DataClass = getattr(medmnist, info['python_class'])
d=DataClass(split='train',download=True)

100%|██████████| 205615438/205615438 [00:07<00:00, 27877060.17it/s]


In [ ]:
type(d)

medmnist.dataset.PathMNIST

In [ ]:
[d[i][1] for i in range(100)]

[array([0]),
 array([4]),
 array([7]),
 array([5]),
 array([5]),
 array([8]),
 array([3]),
 array([3]),
 array([5]),
 array([2]),
 array([8]),
 array([5]),
 array([8]),
 array([2]),
 array([1]),
 array([1]),
 array([5]),
 array([4]),
 array([3]),
 array([2]),
 array([8]),
 array([2]),
 array([5]),
 array([8]),
 array([3]),
 array([8]),
 array([5]),
 array([2]),
 array([3]),
 array([3]),
 array([1]),
 array([3]),
 array([7]),
 array([0]),
 array([1]),
 array([8]),
 array([3]),
 array([0]),
 array([8]),
 array([5]),
 array([4]),
 array([3]),
 array([7]),
 array([1]),
 array([6]),
 array([5]),
 array([6]),
 array([0]),
 array([1]),
 array([2]),
 array([0]),
 array([1]),
 array([0]),
 array([1]),
 array([8]),
 array([4]),
 array([6]),
 array([5]),
 array([1]),
 array([5]),
 array([3]),
 array([7]),
 array([4]),
 array([0]),
 array([0]),
 array([6]),
 array([2]),
 array([7]),
 array([1]),
 array([5]),
 array([6]),
 array([6]),
 array([5]),
 array([8]),
 array([0]),
 array([7]),
 array([2]),

## data processing

In [ ]:
data_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[.5], std=[.5])
])

In [ ]:
def convert_to_rgb(pil_img):

    return pil_img.convert('RGB')


In [ ]:
def process_train_datasets(data_list, split='train',download=True,transform=None):
    all_images = []
    all_labels = []
    current_max_label = 0
    label_mapping = {}

    for data_flag in data_list:
        dataset_info = INFO[data_flag]
        num_labels = len(dataset_info["label"])
        DataClass = getattr(medmnist, dataset_info['python_class'])
        dataset_instance = DataClass(split=split,transform=transform,download=download)

        for i in range(len(dataset_instance)):
            img, label = dataset_instance[i]

            # if grayscale, convert to RGB
            if dataset_info['n_channels'] == 1:
                img = convert_to_rgb(img)

            all_images.append(img)

            # update label id
            if dataset_info["task"]=="multi-label, binary-class":
                one_hot_label = torch.tensor(label)
                label = torch.nonzero(one_hot_label, as_tuple=False).squeeze().tolist()
                for l in label:
                    new_label=current_max_label+l
                    if new_label not in label_mapping:
                        str_label=str(l)
                        label_mapping[new_label] = dataset_info["label"][str_label]
            else:
                new_label=current_max_label+label[0]
                if new_label not in label_mapping:
                    str_label=str(label[0])
                    label_mapping[new_label] = dataset_info["label"][str_label]

            all_labels.append(new_label)
        current_max_label = num_labels + current_max_label
    return all_images, all_labels, label_mapping

In [ ]:
from torch.utils.data import Dataset,DataLoader

class Dataset2D(Dataset):
    def __init__(self, images, labels, transform=data_transform):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

In [ ]:
all_train_images, all_train_labels, label_mapping = process_train_datasets(data_list, split='train',download=True)
all_test_images, all_test_labels, label_mapping_test = process_train_datasets(data_list, split='test',download=True)

Using downloaded and verified file: /home/hanwzhan/.medmnist/pathmnist.npz


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 82802576/82802576 [00:02<00:00, 38572245.52it/s]


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19725078/19725078 [00:00<00:00, 137552660.77it/s]


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 54938180/54938180 [00:01<00:00, 33055820.12it/s]


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4170669/4170669 [00:00<00:00, 58719453.49it/s]


...

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 124962739/124962739 [00:02<00:00, 54070380.09it/s]


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 38247708/38247708 [00:00<00:00, 50538826.00it/s]


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15526411/15526411 [00:00<00:00, 80441134.29it/s]


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 16528359/16528359 [00:00<00:00, 55809920.34it/s]


Using downloaded and verified file: /home/hanwzhan/.medmnist/pathmnist.npz
Using downloaded and verified file: /home/hanwzhan/.medmnist/chestmnist.npz
Using downloaded and verified file: /home/hanwzhan/.medmnist/dermamnist.npz
Using downloaded and verified file: /home/hanwzhan/.medmnist/octmnist.npz
Using downloaded and verified file: /home/hanwzhan/.medmnist/pneumoniamnist.npz
Using downloaded and verified file: /home/hanwzhan/.medmnist/retinamnist.npz
Using downloaded and verified file: /home/hanwzhan/.medmnist/breastmnist.npz
Using downloaded and verified file: /home/hanwzhan/.medmnist/bloodmnist.npz
Using downloaded and verified file: /home/hanwzhan/.medmnist/tissuemnist.npz
Using downloaded and verified file: /home/hanwzhan/.medmnist/organamnist.npz
Using downloaded and verified file: /home/hanwzhan/.medmnist/organcmnist.npz
Using downloaded and verified file: /home/hanwzhan/.medmnist/organsmnist.npz


In [ ]:
label_mapping=dict(sorted(label_mapping.items()))
label_mapping_test=dict(sorted(label_mapping_test.items()))
label_mapping==label_mapping_test

True

In [ ]:
new_old_label_mapping = {original_label: new_label for new_label, original_label in enumerate(sorted(label_mapping.keys()))}

In [ ]:
new_label_mapping = new_labels_list = {new_old_label_mapping[key]: value for key, value in label_mapping.items()}
all_new_train_labels=[new_old_label_mapping[label] for label in all_train_labels]
all_new_test_labels=[new_old_label_mapping[label] for label in all_test_labels]

In [ ]:
BATCH_SIZE = 128

train_dataset=Dataset2D(all_train_images, all_new_train_labels)
test_dataset=Dataset2D(all_test_images, all_new_test_labels)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=2*BATCH_SIZE, shuffle=False)

In [ ]:
n_channels = 3
n_classes = len(label_mapping)
NUM_EPOCHS = 3
lr = 0.001

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


model = ResNet18(in_channels=n_channels, num_classes=n_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)

In [ ]:
print(max(new_label_mapping.keys()))

79


In [ ]:
for epoch in range(NUM_EPOCHS):
    train_correct = 0
    train_total = 0
    test_correct = 0
    test_total = 0

    model.train()
    for inputs, targets in tqdm(train_loader):
        # forward + backward + optimize
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)

        targets = targets.to(torch.long)
        loss = criterion(outputs, targets)


        loss.backward()
        optimizer.step()

  0%|                                                                                                                                       | 0/4049 [00:00<?, ?it/s]/home/hanwzhan/opt/i02/mnlp/lib64/python3.9/site-packages/torch/nn/modules/conv.py:456: UserWarning: Attempt to open cnn_infer failed: handle=0 error: libcudnn_cnn_infer.so.8: cannot open shared object file: No such file or directory (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:78.)
  return F.conv2d(input, weight, bias, self.stride,
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4049/4049 [01:53<00:00, 35.55it/s]


In [ ]:
# evaluation

def test(split):
    model.eval()
    y_true = torch.tensor([])
    y_score = torch.tensor([])

    data_loader = train_loader_at_eval if split == 'train' else test_loader

    with torch.no_grad():
        for inputs, targets in data_loader:
            outputs = model(inputs)

            if task == 'multi-label, binary-class':
                targets = targets.to(torch.float32)
                outputs = outputs.softmax(dim=-1)
            else:
                targets = targets.squeeze().long()
                outputs = outputs.softmax(dim=-1)
                targets = targets.float().resize_(len(targets), 1)

            y_true = torch.cat((y_true, targets), 0)
            y_score = torch.cat((y_score, outputs), 0)

        y_true = y_true.numpy()
        y_score = y_score.detach().numpy()

        evaluator = Evaluator(data_flag, split)
        metrics = evaluator.evaluate(y_score)

        print('%s  auc: %.3f  acc:%.3f' % (split, *metrics))


print('==> Evaluating ...')
test('train')
test('test')